In [62]:
import pandas as pd


## Reading movies and credits dataset from TMDB

In [63]:
credits =pd.read_csv('tmdb_5000_credits.csv')
movies =pd.read_csv('tmdb_5000_movies.csv')

In [64]:
movies.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [65]:
movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

### Selecting the important columns that can be the important factors for recommendation

In [66]:
df =movies[['genres','id','original_language','title','production_companies']].copy()

In [67]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


### we observe that production_companies,cast,crew and genre  are in JSON format
#### checking the JSON string

In [68]:
jstr=str(df['genres'][0])
print(jstr)

[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]


### Now let's see the range of time for all the movies in our dataset.
#### Looks like only till year 2017. we need to add latest movies afterwards.

In [69]:
mx,mn =-9999,9999
for i in movies.release_date:
  if str(i)!='nan':
    val=int(str(i).split('-')[0])
  if mx <val:
    mx=val
  if mn>val:
    mn=val
mx,mn

(2017, 1916)

In [70]:
import json 
final_json =json.loads(jstr)
final_json

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

## Let extract some features and analyse each for our system. 

In [71]:
genre_list=[]                 # list to store genres for each movie
for st in df['genres']:
  fst=json.loads(str(st))    # string to JSON format 
  x=""
  for i in fst:
    x=x+i['name']+" "
  if x!="":
    genre_list.append([x])
  else:
    genre_list.append(['nan'])         # nan for empty value


### Extract the genre's from all the movies 
#### They play a important role finding similarity b/w movies

In [72]:
genre_list[:20]

[['Action Adventure Fantasy Science Fiction '],
 ['Adventure Fantasy Action '],
 ['Action Adventure Crime '],
 ['Action Crime Drama Thriller '],
 ['Action Adventure Science Fiction '],
 ['Fantasy Action Adventure '],
 ['Animation Family '],
 ['Action Adventure Science Fiction '],
 ['Adventure Fantasy Family '],
 ['Action Adventure Fantasy '],
 ['Adventure Fantasy Action Science Fiction '],
 ['Adventure Action Thriller Crime '],
 ['Adventure Fantasy Action '],
 ['Action Adventure Western '],
 ['Action Adventure Fantasy Science Fiction '],
 ['Adventure Family Fantasy '],
 ['Science Fiction Action Adventure '],
 ['Adventure Action Fantasy '],
 ['Action Comedy Science Fiction '],
 ['Action Adventure Fantasy ']]

In [73]:
df['genre_list'] =pd.DataFrame(genre_list,columns=['genre_list'])

In [74]:
df.head()

,genres,id,original_language,title,production_companies,genre_list
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,en,Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Action Adventure Fantasy Science Fiction
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,en,Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Adventure Fantasy Action
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,en,Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Action Adventure Crime
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,en,The Dark Knight Rises,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Action Crime Drama Thriller
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,en,John Carter,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Action Adventure Science Fiction


In [75]:
len(genre_list)

4803

### Now for cast. 
#### Cast is selected because of following reason.
1) generally same cast is for upcoming movies.
2) movies filtered by main cast.

In [76]:
cast_list=[]                    #  list for storing cast
for st in credits['cast']:
  fst=json.loads(str(st))      # string to JSON
  x=""
  for i in fst[0:3]:
    i['character']="".join(i['character'].split(' '))    
    x=x+i['character']+" "
  if x!="":
    cast_list.append([x])
  else:
    cast_list.append(['nan'])


In [77]:
cast_list[:20]

[['JakeSully Neytiri Dr.GraceAugustine '],
 ['CaptainJackSparrow WillTurner ElizabethSwann '],
 ['JamesBond Blofeld Madeleine '],
 ['BruceWayne/Batman AlfredPennyworth JamesGordon '],
 ['JohnCarter DejahThoris Sola '],
 ['PeterParker/Spider-Man MaryJaneWatson HarryOsborn/NewGoblin '],
 ['FlynnRider(voice) Rapunzel(voice) MotherGothel(voice) '],
 ['TonyStark/IronMan ThorOdinson BruceBanner/Hulk '],
 ['HarryPotter RonWeasley HermioneGranger '],
 ['BruceWayne/Batman ClarkKent/Superman DianaPrince/WonderWoman '],
 ['Superman/ClarkKent LexLuthor LoisLane '],
 ['JamesBond CamilleMontes DominicGreene '],
 ['CaptainJackSparrow WillTurner ElizabethSwann '],
 ['Tonto JohnReid/TheLoneRanger ButchCavendish '],
 ['ClarkKent/Kal-El LoisLane GeneralZod '],
 ['PrinceCaspian PeterPevensie SusanPevensie '],
 ['TonyStark/IronMan SteveRogers/CaptainAmerica BruceBanner/TheHulk '],
 ['CaptainJackSparrow AngelicaTeach CaptainEdward"Blackbeard"Teach '],
 ['AgentJ AgentK YoungAgentK '],
 ['BilboBaggins Gandalf

In [78]:
df['cast_list'] =pd.DataFrame(cast_list,columns=['cast_list'])

In [79]:
df.head()

,genres,id,original_language,title,production_companies,genre_list,cast_list
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,en,Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Action Adventure Fantasy Science Fiction,JakeSully Neytiri Dr.GraceAugustine
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,en,Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Adventure Fantasy Action,CaptainJackSparrow WillTurner ElizabethSwann
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,en,Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Action Adventure Crime,JamesBond Blofeld Madeleine
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,en,The Dark Knight Rises,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Action Crime Drama Thriller,BruceWayne/Batman AlfredPennyworth JamesGordon
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,en,John Carter,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Action Adventure Science Fiction,JohnCarter DejahThoris Sola


### Now for crew. 
#### Crew is selected because of following reason.
1) generally same crew (director) are for movies with similar genre.
2) movies filtered by main crew. (preferred)

In [80]:
crew_list=[]                          #list to store crew
for st in credits['crew']:
  fst=json.loads(str(st))
  flag=0
  for i in fst:
    if i['job']=='Director':
      crew_list.append([i['name']])     #getting director name
      flag=1
      break
  if flag==0:
    crew_list.append(['nan'])





In [81]:
crew_list[:20]                       # famous directors

[['James Cameron'],
 ['Gore Verbinski'],
 ['Sam Mendes'],
 ['Christopher Nolan'],
 ['Andrew Stanton'],
 ['Sam Raimi'],
 ['Byron Howard'],
 ['Joss Whedon'],
 ['David Yates'],
 ['Zack Snyder'],
 ['Bryan Singer'],
 ['Marc Forster'],
 ['Gore Verbinski'],
 ['Gore Verbinski'],
 ['Zack Snyder'],
 ['Andrew Adamson'],
 ['Joss Whedon'],
 ['Rob Marshall'],
 ['Barry Sonnenfeld'],
 ['Peter Jackson']]

In [82]:
df['crew_list'] =pd.DataFrame(crew_list,columns=['crew_list'])

In [83]:
df.head()

,genres,id,original_language,title,production_companies,genre_list,cast_list,crew_list
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,en,Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Action Adventure Fantasy Science Fiction,JakeSully Neytiri Dr.GraceAugustine,James Cameron
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,en,Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Adventure Fantasy Action,CaptainJackSparrow WillTurner ElizabethSwann,Gore Verbinski
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,en,Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Action Adventure Crime,JamesBond Blofeld Madeleine,Sam Mendes
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,en,The Dark Knight Rises,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Action Crime Drama Thriller,BruceWayne/Batman AlfredPennyworth JamesGordon,Christopher Nolan
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,en,John Carter,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Action Adventure Science Fiction,JohnCarter DejahThoris Sola,Andrew Stanton


In [84]:
df.isnull().sum()            # no empty values


genres                  0
id                      0
original_language       0
title                   0
production_companies    0
genre_list              0
cast_list               0
crew_list               0
dtype: int64

### Now for companies.
#### Companies are selected because of following reason.
1) Mostly same production company will work for future movies and directors

In [85]:
comp_list=[]           # comp list
for st in movies['production_companies']:
  fst=json.loads(str(st))
  x=""
  for i in fst:
    if i['name']:
      x=x+"".join(i['name'].split(' '))+" "
  comp_list.append([x])

    


In [86]:
comp_list[:20]

[['IngeniousFilmPartners TwentiethCenturyFoxFilmCorporation DuneEntertainment LightstormEntertainment '],
 ['WaltDisneyPictures JerryBruckheimerFilms SecondMateProductions '],
 ['ColumbiaPictures Danjaq B24 '],
 ['LegendaryPictures WarnerBros. DCEntertainment Syncopy '],
 ['WaltDisneyPictures '],
 ['ColumbiaPictures LauraZiskinProductions MarvelEnterprises '],
 ['WaltDisneyPictures WaltDisneyAnimationStudios '],
 ['MarvelStudios PrimeFocus RevolutionSunStudios '],
 ['WarnerBros. HeydayFilms '],
 ['DCComics AtlasEntertainment WarnerBros. DCEntertainment Cruel&UnusualFilms RatPac-DuneEntertainment '],
 ['DCComics LegendaryPictures WarnerBros. BadHatHarryProductions '],
 ['EonProductions '],
 ['WaltDisneyPictures JerryBruckheimerFilms SecondMateProductions '],
 ['WaltDisneyPictures JerryBruckheimerFilms InfinitumNihil SilverBulletProductions(II) BlindWinkProductions ClassicMedia '],
 ['LegendaryPictures WarnerBros. DCEntertainment Syncopy CruelandUnusualFilms '],
 ['WaltDisney WaldenMedia

In [87]:
df['comp_list'] =pd.DataFrame(comp_list,columns=['comp_list'])

In [88]:
df.head()

,genres,id,original_language,title,production_companies,genre_list,cast_list,crew_list,comp_list
0,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",19995,en,Avatar,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",Action Adventure Fantasy Science Fiction,JakeSully Neytiri Dr.GraceAugustine,James Cameron,IngeniousFilmPartners TwentiethCenturyFoxFilmC...
1,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",285,en,Pirates of the Caribbean: At World's End,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...",Adventure Fantasy Action,CaptainJackSparrow WillTurner ElizabethSwann,Gore Verbinski,WaltDisneyPictures JerryBruckheimerFilms Secon...
2,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",206647,en,Spectre,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...",Action Adventure Crime,JamesBond Blofeld Madeleine,Sam Mendes,ColumbiaPictures Danjaq B24
3,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",49026,en,The Dark Knight Rises,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...",Action Crime Drama Thriller,BruceWayne/Batman AlfredPennyworth JamesGordon,Christopher Nolan,LegendaryPictures WarnerBros. DCEntertainment ...
4,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",49529,en,John Carter,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]",Action Adventure Science Fiction,JohnCarter DejahThoris Sola,Andrew Stanton,WaltDisneyPictures


#### dropping JSON data

In [89]:
df.drop(['genres','production_companies'],inplace=True,axis=1)

In [90]:
df.head()


,id,original_language,title,genre_list,cast_list,crew_list,comp_list
0,19995,en,Avatar,Action Adventure Fantasy Science Fiction,JakeSully Neytiri Dr.GraceAugustine,James Cameron,IngeniousFilmPartners TwentiethCenturyFoxFilmC...
1,285,en,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,CaptainJackSparrow WillTurner ElizabethSwann,Gore Verbinski,WaltDisneyPictures JerryBruckheimerFilms Secon...
2,206647,en,Spectre,Action Adventure Crime,JamesBond Blofeld Madeleine,Sam Mendes,ColumbiaPictures Danjaq B24
3,49026,en,The Dark Knight Rises,Action Crime Drama Thriller,BruceWayne/Batman AlfredPennyworth JamesGordon,Christopher Nolan,LegendaryPictures WarnerBros. DCEntertainment ...
4,49529,en,John Carter,Action Adventure Science Fiction,JohnCarter DejahThoris Sola,Andrew Stanton,WaltDisneyPictures


In [91]:
df.to_csv('first.csv',index=False)

In [92]:
df_comb =df.copy()

In [93]:
df_comb

,id,original_language,title,genre_list,cast_list,crew_list,comp_list
0,19995,en,Avatar,Action Adventure Fantasy Science Fiction,JakeSully Neytiri Dr.GraceAugustine,James Cameron,IngeniousFilmPartners TwentiethCenturyFoxFilmC...
1,285,en,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,CaptainJackSparrow WillTurner ElizabethSwann,Gore Verbinski,WaltDisneyPictures JerryBruckheimerFilms Secon...
2,206647,en,Spectre,Action Adventure Crime,JamesBond Blofeld Madeleine,Sam Mendes,ColumbiaPictures Danjaq B24
3,49026,en,The Dark Knight Rises,Action Crime Drama Thriller,BruceWayne/Batman AlfredPennyworth JamesGordon,Christopher Nolan,LegendaryPictures WarnerBros. DCEntertainment ...
4,49529,en,John Carter,Action Adventure Science Fiction,JohnCarter DejahThoris Sola,Andrew Stanton,WaltDisneyPictures
...,...,...,...,...,...,...,...
4798,9367,es,El Mariachi,Action Crime Thriller,ElMariachi Bigotón Mauricio(Moco),Robert Rodriguez,ColumbiaPictures
4799,72766,en,Newlyweds,Comedy Romance,Buzzy Linda Marsha,Edward Burns,
4800,231617,en,"Signed, Sealed, Delivered",Comedy Drama Romance TV Movie,OliverO’Toole ShaneMcInerney RitaHaywith,Scott Smith,FrontStreetPictures MuseEntertainmentEnterprises
4801,126186,en,Shanghai Calling,nan,Sam Amanda Donald,Daniel Hsia,


### Combining columns to use a single string. It will help to produce vector easily further for similarity

In [94]:
df_comb['comb'] =df_comb['genre_list']+df_comb['cast_list']+df_comb['crew_list']+df_comb['comp_list']+df_comb['original_language']

## We will use WORD to VEC for converting words to vectors and the applying cosine similarity

In [95]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


In [96]:
cv=CountVectorizer()
count_matrix = cv.fit_transform(df_comb['comb'])

In [97]:
similarity =cosine_similarity(count_matrix)

## Finally Let's built a function for recommending movies using similarity

In [98]:
def recommend(movie):
  i=df_comb.loc[df_comb['title']==movie].index[0]
  List =list(enumerate(similarity[i]))
  List_sorted =sorted(List,reverse=True,key=lambda x:x[1])
  Final_list=List_sorted[1:15]        # 1:15 because 0 will be the movie itself because of 1 correlation.
  recommend_list =[]
  for i in range(len(Final_list)):
    a=Final_list[i][0]
    recommend_list.append(df_comb['title'][a])

  return recommend_list


#### Let's  see for superman Returns 

In [99]:
my_movies = recommend("Superman Returns")
for m in my_movies:
  print(m)

Superman IV: The Quest for Peace
Superman II
Superman
Man of Steel
Superman III
Jack the Giant Slayer
Jupiter Ascending
The Time Machine
Beastmaster 2: Through the Portal of Time
Mad Max: Fury Road
The Shadow
Suicide Squad
Batman v Superman: Dawn of Justice
The Wolverine


### It works!!!!! 
### Now Let's check for Pirates of the Caribbean: At World's End

In [100]:
my_movies = recommend("Pirates of the Caribbean: At World's End")
for m in my_movies:
  print(m)

Pirates of the Caribbean: Dead Man's Chest
Pirates of the Caribbean: The Curse of the Black Pearl
Pirates of the Caribbean: On Stranger Tides
Prince of Persia: The Sands of Time
The Lone Ranger
The Barbarians
Thor: The Dark World
Thor
The 13th Warrior
Conan the Destroyer
The Sorcerer's Apprentice
The Lord of the Rings: The Return of the King
Conan the Barbarian
Cirque du Freak: The Vampire's Assistant


## great accuracy

In [101]:
df_comb.to_csv('final_movie_data.csv',index=False)